# Implementation of N-Gram

Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import re
import nltk
from nltk.util import ngrams
import itertools
import urllib
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
import string
N_for_NGram = 3
Sequence_length = 300
Batch_Size = 64
epochs = 5
n_classes = 2

In [2]:
# Define a function to preprocess the link
def preprocess_link(link):
    
    # Convert all letters to lowercase
    link = link.lower()
    
    link = link.replace("http://","")
    link = link.replace("https://","")
    link = link.replace("www.","")
    link = link.replace(" ","")
    link = link.strip()
    
    # Remove punctuation and special characters
    link = re.sub('[^A-Za-z0-9]+', '', link)
    
    return link

In [3]:
from sklearn.model_selection import train_test_split
url_dataframe = pd.read_csv("../dataset/df_final.csv")
url_dataframe['url'] = url_dataframe['url'].apply(preprocess_link)
url_dataframe = url_dataframe.drop_duplicates(subset=['url'])
url_dataframe = url_dataframe.dropna().reset_index(drop=True)
url_dataframe

,url,type
0,extremetubecomvideobittitblondeslutinglassesfu...,1
1,playarprintcomchasehomephppublicenrollidentify...,0
2,genforumgenealogycommcgeepage15html,1
3,itunesapplecomgbalbumwhbbonustrackversionid358...,1
4,holdingmaclincleieindexphp,0
...,...,...
770410,facebookcompagesdrstevebrule31743261441,1
770411,absoluteastronomycomtopicslistoffamilyrelation...,1
770412,oceanpacificsclmainhtm,1
770413,viewpointscomsamuelzarkoffcollectioncultclassi...,1


Creating N-Gram Vectors set

In [4]:
x = url_dataframe['url'].str.len()
x.describe(percentiles=[0.25,0.5,0.75,0.9,0.98,0.99,0.9999,0.99999,1])

count      770415.000000
mean           44.570860
std            35.836788
min             0.000000
25%            22.000000
50%            34.000000
75%            56.000000
90%            80.000000
98%           145.000000
99%           189.000000
99.99%        573.000000
99.999%      1272.000000
100%         2141.000000
max          2141.000000
Name: url, dtype: float64

In [5]:
import tensorflow as tf
train_df, test_df = train_test_split(url_dataframe, test_size=0.20)

2023-03-04 00:13:44.555351: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-04 00:13:44.673607: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-04 00:13:44.673629: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-04 00:13:44.695908: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-03-04 00:13:45.218064: W tensorflow/stream_executor/platform/de

In [6]:
train_ds = tf.data.Dataset.from_tensor_slices((train_df['url'].values, train_df['type'].values))
test_ds = tf.data.Dataset.from_tensor_slices((test_df['url'].values, test_df['type'].values))

2023-03-04 00:13:45.998354: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-04 00:13:45.998384: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-04 00:13:45.998401: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (munna0912-VivoBook-ASUSLaptop-X421IAY-M413IA): /proc/driver/nvidia/version does not exist
2023-03-04 00:13:45.998687: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
train_ds = train_ds.batch(Batch_Size)
test_ds = test_ds.batch(Batch_Size)

In [8]:
Vectorize_Layer = tf.keras.layers.TextVectorization(standardize='lower_and_strip_punctuation',
                                                   split="character",
                                                   ngrams=(N_for_NGram,),
                                                    max_tokens = 30000,
                                                   output_mode='int',
                                                   output_sequence_length=Sequence_length)

In [9]:
# Make a text-only dataset (without labels), then call adapt
train_text = train_ds.map(lambda x, y: x)
Vectorize_Layer.adapt(train_text)

In [10]:
max_tokens = len(Vectorize_Layer.get_vocabulary())
max_tokens = 5000
# have to tune max_tokens

In [11]:
Vectorize_Layer.get_vocabulary()

['',
 '[UNK]',
 'c o m',
 'h t m',
 'i n d',
 'n d e',
 't m l',
 'e n t',
 'p h p',
 'i o n',
 'd e x',
 's c o',
 't i o',
 'e x p',
 'e c o',
 'n c o',
 'x p h',
 'a r t',
 'o m p',
 'i n g',
 'o n t',
 'i t e',
 'o r g',
 'o n c',
 'c o n',
 'v i e',
 'a g e',
 'n t e',
 'l o g',
 'o m c',
 'e r s',
 'n e t',
 'a t e',
 't o r',
 'a t i',
 't e r',
 'o z i',
 'i e w',
 'm o z',
 't i c',
 'l i n',
 't h e',
 'e s t',
 'a n d',
 't e m',
 'r t i',
 'e d i',
 'z i m',
 'm c o',
 't e n',
 's t a',
 'p t i',
 'e m i',
 '2 0 1',
 't c o',
 'o m a',
 'o p t',
 'a l l',
 'i n e',
 'l c o',
 'm i n',
 'c a t',
 'm i d',
 'i s t',
 'o n e',
 'o m i',
 'f o r',
 'g i n',
 'i k i',
 'p r o',
 'r e s',
 'c l e',
 'w i k',
 'p o p',
 'o m e',
 's t o',
 'i c l',
 'm a r',
 'e s s',
 'h p o',
 'a c o',
 'e s c',
 'p a g',
 'n e w',
 'd i a',
 's t e',
 't i n',
 'o r t',
 'i d 1',
 'y c o',
 'a m e',
 'w a r',
 'm e n',
 'r e a',
 'i n s',
 'a m p',
 'o m s',
 'e n e',
 'o g i',
 'o c o',
 'o u

In [12]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return Vectorize_Layer(text), label

In [13]:
train_ds = train_ds.map(vectorize_text)
test_ds = test_ds.map(vectorize_text)

In [14]:
Vectorize_Layer("hellobrother.com")

<tf.Tensor: shape=(50,), dtype=int64, numpy=
array([ 800,  213,  816, 2015, 3585,  569, 1343, 1018,   41,  217,  126,
        102,    2,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0])>

In [15]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [16]:
model = tf.keras.Sequential([
  tf.keras.layers.Embedding(max_tokens, 64, input_length=Sequence_length),
  tf.keras.layers.LSTM(64, return_sequences=True) ,
  tf.keras.layers.LSTM(64,) ,
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1, activation='sigmoid')])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 64)            320000    
                                                                 
 lstm (LSTM)                 (None, 50, 64)            33024     
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 390,273
Trainable params: 390,273
Non-trai

In [17]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [18]:
history = model.fit(
    train_ds,
    epochs=epochs)

Epoch 1/5
9631/9631 [==============================] - 336s 35ms/step - loss: 0.2337 - accuracy: 0.8974
Epoch 2/5
9631/9631 [==============================] - 350s 36ms/step - loss: 0.1738 - accuracy: 0.9272
Epoch 3/5
9631/9631 [==============================] - 350s 36ms/step - loss: 0.1478 - accuracy: 0.9390
Epoch 4/5
9631/9631 [==============================] - 351s 36ms/step - loss: 0.1301 - accuracy: 0.9466
Epoch 5/5
9631/9631 [==============================] - 361s 37ms/step - loss: 0.1160 - accuracy: 0.9530


In [19]:
import tensorflow as tf
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# Define the evaluation function
def evaluate_model(model, test_dataset):
    y_true = []
    for x, y in test_dataset:
        for yi in y:
            y_true.append(yi.numpy())
    
    y_true = np.array(y_true)
    y_true = y_true.flatten()
    y_pred = model.predict(test_dataset)
    y_pred = y_pred.flatten()
    y_pred = [round(i) for i in y_pred]
    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    print("Confusion Matrix:")
    print(cm)

    # Accuracy
    acc = accuracy_score(y_true, y_pred)
    print("Accuracy:", acc)

    # Precision
    prec = precision_score(y_true, y_pred, average='weighted')
    print("Precision:", prec)

    # Recall
    rec = recall_score(y_true, y_pred, average='weighted')
    print("Recall:", rec)

    # F1 Score
    f1 = f1_score(y_true, y_pred, average='weighted')
    print("F1 Score:", f1)
    
    return [cm,acc,prec,rec,f1]
# Evaluate the model
evaluate_model(model, test_ds)


2408/2408 [==============================] - 34s 14ms/step
Confusion Matrix:
[[63729  6271]
 [ 3757 80326]]
Accuracy: 0.9349181934411973
Precision: 0.935191404475249
Recall: 0.9349181934411973
F1 Score: 0.9348024721272894


[array([[63729,  6271],
        [ 3757, 80326]]),
 0.9349181934411973,
 0.935191404475249,
 0.9349181934411973,
 0.9348024721272894]

In [20]:
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import random
# import re
# from nltk.util import ngrams
# import itertools
# N_for_NGram = 3
# Sequence_length = 200
# Batch_Size = 64
# epochs = 5
# n_classes = 2
# from sklearn.model_selection import train_test_split
# url_dataframe = pd.read_csv("../dataset/df_final.csv")
# url_dataframe['url'] = url_dataframe['url'].str.replace("https://","")
# url_dataframe['url'] = url_dataframe['url'].str.replace("http://","")
# url_dataframe['url'] = url_dataframe['url'].str.replace(" ","")
# url_dataframe['url'] = url_dataframe['url'].str.replace("www","")
# url_dataframe['url'] = url_dataframe['url'].str.strip()
# print(url_dataframe)

# import tensorflow as tf
# train_df, test_df = train_test_split(url_dataframe, test_size=0.20)

# train_ds = tf.data.Dataset.from_tensor_slices((train_df['url'].values, train_df['type'].values))
# test_ds = tf.data.Dataset.from_tensor_slices((test_df['url'].values, test_df['type'].values))

# train_ds = train_ds.batch(Batch_Size)
# test_ds = test_ds.batch(Batch_Size)

# Vectorize_Layer = tf.keras.layers.TextVectorization(standardize='lower_and_strip_punctuation',
#                                                    split="character",
#                                                    ngrams=(N_for_NGram,),
#                                                    output_mode='int',
#                                                    output_sequence_length=Sequence_length)

# # Make a text-only dataset (without labels), then call adapt
# train_text = train_ds.map(lambda x, y: x)
# Vectorize_Layer.adapt(train_text)

# max_tokens = len(Vectorize_Layer.get_vocabulary())
# print("max_tokens" = max_tokens)

# print(Vectorize_Layer.get_vocabulary())

# def vectorize_text(text, label):
#   text = tf.expand_dims(text, -1)
#   return Vectorize_Layer(text), label

# train_ds = train_ds.map(vectorize_text)
# test_ds = test_ds.map(vectorize_text)

# AUTOTUNE = tf.data.AUTOTUNE

# train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
# test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

# model = tf.keras.Sequential([
#   tf.keras.layers.Embedding(max_tokens, 64, input_length=Sequence_length),
#   tf.keras.layers.LSTM(64, return_sequences=True) ,
#   tf.keras.layers.LSTM(64,) ,
#   tf.keras.layers.Dense(64, activation='relu'),
#   tf.keras.layers.Dropout(0.2),
#   tf.keras.layers.Dense(1, activation='sigmoid')])

# print(model.summary())

# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# history = model.fit(
#     train_ds,
#     epochs=epochs)

# from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# # Define the evaluation function
# def evaluate_model(model, test_dataset):
#     y_true = []
#     for x, y in test_dataset:
#         for yi in y:
#             y_true.append(yi.numpy())
    
#     y_true = np.array(y_true)
#     y_true = y_true.flatten()
#     y_pred = model.predict(test_dataset)
#     y_pred = y_pred.flatten()
#     y_pred = [round(i) for i in y_pred]
#     print(y_true)
#     print(y_pred)
#     # Confusion matrix
#     cm = confusion_matrix(y_true, y_pred)
#     print("Confusion Matrix:")
#     print(cm)

#     # Accuracy
#     acc = accuracy_score(y_true, y_pred)
#     print("Accuracy:", acc)

#     # Precision
#     prec = precision_score(y_true, y_pred, average='weighted')
#     print("Precision:", prec)

#     # Recall
#     rec = recall_score(y_true, y_pred, average='weighted')
#     print("Recall:", rec)

#     # F1 Score
#     f1 = f1_score(y_true, y_pred, average='weighted')
#     print("F1 Score:", f1)
    
#     return [cm,acc,prec,rec,f1]
# # Evaluate the model
# result = evaluate_model(model, test_ds)

# import pickle
# with open("model.pkl", "wb") as f:
#     pickle.dump(model, f)
    
# with open("result.pkl", "wb") as f:
#     pickle.dump(result, f)